## Conditional Random Field Model (Window 2)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from collections import OrderedDict

#Custom models
from prepro import readfile, get_sentence, is_number, extract_words,get_label,partial_tags

#Model
from sklearn_crfsuite import CRF

#Evalulation
from sklearn.model_selection import cross_val_predict
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn_crfsuite.metrics import flat_f1_score

In [2]:
#import data from my github repo
test = readfile("test.txt")
train =readfile("train.txt")

# filenames = ['train.txt', 'valid.txt']
# with open('Combined.txt', 'w') as outfile:
#     for fname in filenames:
#         with open(fname) as infile:
#             for line in infile:
#                 outfile.write(line)
                
# train =readfile("Combined.txt")

## Orthographic Feature Engineering

In [3]:
#Special Features:
def count_vowel(word):
    '''
    Function returns the number of vowels in token
    '''
    return sum(list(map(word.lower().count, "aeiou")))

def dash(word):
    '''
    The Function returns whether or not the token contains a dash
    '''
    return 1 if "-" in word else 0

def count_consonants(word):
    '''
    The Function returns the number of consonants in a token
    '''
    vowels="aeiou"
    return sum(i not in vowels for i in word)

def contain_punct(word):
    '''
    The Function returns the boolean if punctuations is present in token
    '''
    if re.match(r'^\w+$',word):return False
    else: return True
    
def apostrophe(word):
    '''
    The Function returns boolean if "'s" is present in token 
    '''
    if word =="'s":return True
    else:return False
    
def word_pattern(word):
    '''
    The Function returns word patter feature
    Upper Case = "A"
    Lower Case = "a"
    Digit = "0"
    '''
    token=""
    for i in word:
        if i.isupper():
            token +="A"
        elif i.islower():
            token +="a"
        elif i.isdigit():
            token +="0"
        else:
            token +=str(i)
    return token

def pattern_sum(word):
    '''
    The Function returns the word patern without consectutive duplicates
    '''
    return ''.join(OrderedDict.fromkeys(word))

In [4]:
def word2features(sentence,i):
    word = sentence[i]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'Prefix_2': word[:2], # prefix
        'Prefix_3': word[:3], # prefix
        
        'word.isupper()': word.isupper(), #all caps
        'word.islower()': word.islower(),# all lower case
        'word.istitle()': word.istitle(), #starts with caps
        'word.isdigit()': word.isdigit(), #all digits
        'word.isalpha()': word.isalpha(), #all letters
        'word.isalnum()': word.isalnum(), #mixture of letters and digits
        
        'word.pattern()': word_pattern(word),#word pattern
        'word.pattern_sum()': pattern_sum(word_pattern(word)),
        'word.punct()':contain_punct(word),#contains punctuation
        'word.apost()':apostrophe(word), #is an apostrophe
        
        'Suffix_2': word[-2:], # suffix
        'Suffix_3': word[-3:], # suffix
    }
    if  i == 1 :
        '''
        Get the token before in the sentence
        '''
        word1 = sentence[i-1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word Prefix_2': word1[:2],
            '-1:word Prefix_3': word1[:3], 
            
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.islower()': word1.lower(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.isalpha()': word1.isalpha(),
            '-1:word.isalnum()': word1.isalnum(),
            
            '-1:word.pattern()': word_pattern(word1),
            '-1:word.pattern_sum()': pattern_sum(word_pattern(word1)),
            '-1:word.punct()':contain_punct(word1),
            '-1:word.apost()':apostrophe(word1),
            
            '-1:word.Suffix_2': word1[-2:],
            '-1:word.Suffix_3': word1[-3:],
        })
        
    elif i>1 :
        word1 = sentence[i-1]
        word2 = sentence[i-2]
        
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word Prefix_2': word1[:2],
            '-1:word Prefix_3': word1[:3], 
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.islower()': word1.lower(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.isalpha()': word1.isalpha(),
            '-1:word.isalnum()': word1.isalnum(),
            '-1:word.pattern()': word_pattern(word1),
            '-1:word.pattern_sum()': pattern_sum(word_pattern(word1)),
            '-1:word.punct()':contain_punct(word1),
            '-1:word.apost()':apostrophe(word1),
            '-1:word.Suffix_2': word1[-2:],
            '-1:word.Suffix_3': word1[-3:],
            
            
            
            '-2:word.lower()': word2.lower(),
            '-2:word Prefix_2': word2[:2],
            '-2:word Prefix_3': word2[:3], 
            '-2:word.istitle()': word2.istitle(),
            '-2:word.isupper()': word2.isupper(),
            '-2:word.islower()': word2.lower(),
            '-2:word.isdigit()': word2.isdigit(),
            '-2:word.isalpha()': word2.isalpha(),
            '-2:word.isalnum()': word2.isalnum(),
            '-2:word.pattern()': word_pattern(word2),
            '-2:word.pattern_sum()': pattern_sum(word_pattern(word2)),
            '-2:word.punct()':contain_punct(word2),
            '-2:word.apost()':apostrophe(word2),
            '-2:word.Suffix_2': word2[-2:],
            '-2:word.Suffix_3': word2[-3:],
        })
        
    else:
        features['Start'] = True
        
    if i == len(sentence)-2:
        '''
        Get the token after in the sentence
        '''
        word1 = sentence[i+1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word Prefix_2': word1[:2],
            '+1:word Prefix_3': word1[:3], 
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.islower()': word1.lower(),
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.isalpha()': word1.isalpha(),
            '+1:word.isalnum()': word1.isalnum(),
            '+1:word.pattern()': word_pattern(word1),
            '+1:word.pattern_sum()': pattern_sum(word_pattern(word1)),
            '+1:word.punct()':contain_punct(word1),
            '+1:word.apost()':apostrophe(word1),
            '+1:word.Suffix_2': word1[-2:],
            '+1:word.Suffix_3': word1[-3:],
        })
        
    elif i < len(sentence)-2:
        
        word1 = sentence[i+1]
        word2 = sentence[i+2]
        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word Prefix_2': word1[:2],
            '+1:word Prefix_3': word1[:3], 
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.islower()': word1.lower(),
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.isalpha()': word1.isalpha(),
            '+1:word.isalnum()': word1.isalnum(),
            '+1:word.pattern()': word_pattern(word1),
            '+1:word.pattern_sum()': pattern_sum(word_pattern(word1)),
            '+1:word.punct()':contain_punct(word1),
            '+1:word.apost()':apostrophe(word1),
            '+1:word.Suffix_2': word1[-2:],
            '+1:word.Suffix_3': word1[-3:],
            
            '+2:word.lower()': word2.lower(),
            '+2:word Prefix_2': word2[:2],
            '+2:word Prefix_3': word2[:3], 
            '+2:word.istitle()': word2.istitle(),
            '+2:word.isupper()': word2.isupper(),
            '+2:word.islower()': word2.lower(),
            '+2:word.isdigit()': word2.isdigit(),
            '+2:word.isalpha()': word2.isalpha(),
            '+2:word.isalnum()': word2.isalnum(),
            '+2:word.pattern()': word_pattern(word2),
            '+2:word.pattern_sum()': pattern_sum(word_pattern(word2)),
            '+2:word.punct()':contain_punct(word2),
            '+2:word.apost()':apostrophe(word2),
            '+2:word.Suffix_2': word2[-2:],
            '+2:word.Suffix_3': word2[-3:],
        })
        
    else:
        features['End'] = True
    
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def get_all_sentences(dataset):
    sentences=[]
    for i in range(len(dataset)):
        sentences.append(get_sentence(dataset,i+1))
    return sentences

def get_all_labels(dataset):
    labels=[]
    for i in range(len(dataset)):
        #labels.append(partial_tags(get_label(dataset,i+1)))
        labels.append(get_label(dataset,i+1))
    return labels

In [5]:
#get the first sentence
sent = get_sentence(train,1)
#label = partial_tags(get_label(train,1))
label = get_label(train,1)

#print out first sentence
print(sent)
print(label)

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']
['I-ORG', 'O', 'I-MISC', 'O', 'O', 'O', 'I-MISC', 'O', 'O']


In [6]:
#Applt feature engineering
train_sents = get_all_sentences(train)
train_labels = get_all_labels(train)
test_sents = get_all_sentences(test)
test_labels = get_all_labels(test)

X_train = [sent2features(s) for s in train_sents]
y_train = train_labels
X_test = [sent2features(s) for s in test_sents]
y_test = test_labels


sub_labels=list(set([item for sublist in train_labels for item in sublist]))
sub_labels.remove("O")


## Condtional Random Field Model

### Model 1
- Using Stochastic Gradient descent 

In [9]:
%%time
crf4 = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.2,
          max_iterations=50,
          all_possible_transitions=False)

#training model
crf4.fit(X=X_train, y=y_train)

#generate predictions
pred = crf4.predict(X_train)

#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_train,labels=sub_labels)
print(report)

score=flat_f1_score(y_pred=pred, y_true=y_train,average='micro',labels=sub_labels)
print(score)

              precision    recall  f1-score   support

      B-MISC       0.97      0.89      0.93        37
       B-LOC       1.00      1.00      1.00        11
      I-MISC       1.00      1.00      1.00      4556
       I-ORG       1.00      1.00      1.00     10001
       B-ORG       1.00      1.00      1.00        24
       I-LOC       1.00      1.00      1.00      8286
       I-PER       1.00      1.00      1.00     11128

   micro avg       1.00      1.00      1.00     34043
   macro avg       1.00      0.98      0.99     34043
weighted avg       1.00      1.00      1.00     34043

0.9996034135808817
Wall time: 48.9 s


# Predict on Test dataset

In [10]:
#prediction with best performaning model
pred = crf4.predict(X_test)

#generate report on entire model
report = flat_classification_report(y_pred=pred, y_true=y_test,labels=sub_labels)
print(report)

score=flat_f1_score(y_pred=pred, y_true=y_test,average='micro',labels=sub_labels)
print(score)

C:\Users\dhoperoberts\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

      B-MISC       0.00      0.00      0.00         9
       B-LOC       0.00      0.00      0.00         6
      I-MISC       0.75      0.78      0.76       909
       I-ORG       0.81      0.80      0.81      2491
       B-ORG       0.00      0.00      0.00         5
       I-LOC       0.85      0.87      0.86      1919
       I-PER       0.89      0.89      0.89      2773

   micro avg       0.84      0.85      0.84      8112
   macro avg       0.47      0.48      0.47      8112
weighted avg       0.84      0.85      0.84      8112

0.8432179952061951
